In [ ]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://es:9200")

In [ ]:
index = 'test'
try:
    es.indices.delete(index)
except:
    pass
es.indices.create(index)

In [ ]:
mapping = {
  "properties": {
    "file_id": {
      "type": "keyword"
    },
    "location": {
      "type": "geo_point"
    },
    "date": {
      "type": "date"
    },
    "qa": {
        "type": "short"
    },
    "value": {
        "type": "integer"
    }
  }
}

es.indices.put_mapping(index=index, body=mapping)

In [ ]:
es.indices.get_mapping(index)

In [ ]:
from elasticsearch import helpers

def gendata(index, items):
    for item in items:
        yield {
            "_op_type": "create",
            "_index": index,
            "_source": item
        }

items = [
    {"file_id": "Tokyo", "location": [139.74730, 35.68464], "date": "2014-01-01", "qa": [1], "value": 28},
    {"file_id": "Tokyo", "location": [139.44798, 35.69978], "date": "2015-01-01", "qa": [0, 1], "value": 24},
    {"file_id": "Tokyo", "location": [139.45015, 35.69464], "date": "2016-01-01", "qa": [2], "value": 25},
    {"file_id": "Hokkaido", "location": [142.51199, 42.81914], "date": "2013-01-01", "value": 18},
    {"file_id": "Osaka", "location": [135.60643, 34.76895], "date": "2012-01-01", "qa": [1], "value": 26},
    {"file_id": "Aichi", "location": [136.88158, 35.17095], "date": "2015-01-01", "qa": [0, 1, 2], "value": 25},
    {"file_id": "Fukuoka", "location": [130.36511, 33.57753], "date": "2016-01-01", "qa": [0], "value": 27}
]

helpers.bulk(es, gendata(index, items))

In [ ]:
es.search(index=index)

In [ ]:
query = {
   "query":{
      "bool":{
         "must":{
            "match_all":{
               
            }
         },
         "filter":{
            "geo_distance":{
               "distance":"600km",
               "location":[139.45, 35.695]
            }
         }
      }
   }
}

es.search(index=index, body=query)

In [ ]:
query = {
   "query":{
      "bool":{
         "must":{
            "range": {
              "date": {
                "gte": "2013-01-01", 
                "lt": "2016-01-01",
                "time_zone": "+00:00"
              }
            }
         }
      }
   }
}

es.search(index=index, body=query)

In [ ]:
query = {
   "query":{
      "bool":{
         "must": [
            {
                "match": {
                  "qa": 1
                }
            },
            {
                "match": {
                  "qa": 0
                }
            }
         ]
      }
   }
}

es.search(index=index, body=query)

In [ ]:
query = {
   "query":{
      "bool":{
         "should": [
            {
                "match": {
                  "qa": 1
                }
            },
            {
                "match": {
                  "qa": 0
                }
            }
         ],
          "must_not": [
            {
                "match": {
                  "qa": 2
                }
            }
          ]
      }
   }
}

es.search(index=index, body=query)

In [ ]:
query = {
   "query":{
      "bool":{
         "must_not": [
            {
                "match": {
                  "qa": 2
                }
            }
         ],
         "filter":{
            "geo_distance":{
               "distance":"600km",
               "location":[139.45, 35.695]
            }
         }
      }
   },
   "sort":[
      {
         "date":{
            "order":"asc"
         }
      }
   ],
   "collapse":{
      "field":"file_id",
      "inner_hits":{
         "name":"distance",
         "size":5,
         "sort":[
            {
               "_geo_distance":{
                  "location":{
                     "lat":35.6970,
                     "lon":139.7735
                  }
               }
            }
         ]
      }
   }
}
es.search(index=index, body=query)